In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
train_df["planArrivalNumeric"] = train_df["planArrival"].map(lambda x : int(x.replace(":", "")))
train_df

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,planArrivalNumeric
0,0,20191204,A,1,AFGKvJ,Rlfq,06:11,0.0,611
1,1,20191204,A,1,AFGKvJ,coZB,06:13,0.0,613
2,2,20191204,A,1,AFGKvJ,LMww,06:16,0.0,616
3,3,20191204,A,1,AFGKvJ,VNyR,06:18,0.0,618
4,4,20191204,A,1,AFGKvJ,jhlV,06:20,0.0,620
...,...,...,...,...,...,...,...,...,...
1488880,1488880,20201022,D,2,zbKwea,BCRD,18:47,0.0,1847
1488881,1488881,20201022,D,2,zbKwea,AVjc,18:51,1.0,1851
1488882,1488882,20201022,D,2,zbKwea,uYlv,18:53,0.0,1853
1488883,1488883,20201022,D,2,zbKwea,wwYD,18:59,0.0,1859


In [2]:
# testと同じ状況にするためにnanを作らなきゃいけない
# とりあえず完璧に存在しない時間帯は消す
train_index_am = train_df.query("901 <= planArrivalNumeric <= 1200").index
train_index_pm = train_df.query("1901 <= planArrivalNumeric").index

#print(train_df.loc[train_index_am, "delayTime"].shape)
train_df.loc[train_index_am, "delayTime"] = np.nan
train_df.loc[train_index_pm, "delayTime"] = np.nan

#一部存在する日付時間帯の一部以外を消す
date_am = test_df[test_df["planArrival"] == "08:01"].groupby("date").agg({"delayTime" : "count"}).query("delayTime != 0").index.to_list()
train_df_am = train_df.query("801 <= planArrivalNumeric <= 900")
train_index_am = train_df_am[~train_df_am["date"].isin(date_am)].index
print(len(train_index_am))

date_pm = test_df[test_df["planArrival"] == "18:01"].groupby("date").agg({"delayTime" : "count"}).query("delayTime != 0").index.to_list()
train_df_pm = train_df.query("1801 <= planArrivalNumeric <= 1900")
train_index_pm = train_df_pm[~train_df_pm["date"].isin(date_pm)].index
print(len(train_index_pm))

train_df.loc[train_index_am, "delayTime"] = np.nan
train_df.loc[train_index_pm, "delayTime"] = np.nan


train_df.count()

238144
195110


id                    1488885
date                  1488885
lineName              1488885
directionCode         1488885
trainNo               1488885
stopStation           1488885
planArrival           1488885
delayTime              527527
planArrivalNumeric    1488885
dtype: int64

In [3]:
#trandform は圧縮しない
# mergeの手間が省ける
train_df["firstDelayTime"] = train_df.groupby(["date", "trainNo"])["delayTime"].transform(lambda x: x.fillna(method="ffill").fillna(0))
test_df["firstDelayTime"] = test_df.groupby(["date", "trainNo"])["delayTime"].transform(lambda x: x.fillna(method="ffill").fillna(0))

In [5]:
train_df[:100]

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,planArrivalNumeric,firstDelayTime
0,0,20191204,A,1,AFGKvJ,Rlfq,06:11,0.0,611,0.0
1,1,20191204,A,1,AFGKvJ,coZB,06:13,0.0,613,0.0
2,2,20191204,A,1,AFGKvJ,LMww,06:16,0.0,616,0.0
3,3,20191204,A,1,AFGKvJ,VNyR,06:18,0.0,618,0.0
4,4,20191204,A,1,AFGKvJ,jhlV,06:20,0.0,620,0.0
5,5,20191204,A,1,AFGKvJ,efzW,06:22,0.0,622,0.0
6,6,20191204,A,1,AFGKvJ,PcxI,06:25,0.0,625,0.0
7,7,20191204,A,1,AFGKvJ,ejfb,06:27,0.0,627,0.0
8,8,20191204,A,1,AFGKvJ,RDLf,06:28,0.0,628,0.0
9,9,20191204,A,1,AFGKvJ,cRgf,06:31,0.0,631,0.0


In [8]:
test_df[["id"]] is not None

True

In [ ]:
"""
7:00 ~ 8:00 testデータでも完全にデータが与えられている
8:01 ~ 9:00 わずかに与えられているデータがある #
9:01 ~ 11:04 全く与えられていない

17:00 ~ 18:00 全て与えられている
18:01 ~ 19:00 与えられている区間もある #
19:01 ~ 22:05 全て与えられていない

#の時間帯は、ある日付だけ与えられている？
"""
test_df.groupby("planArrival").agg({"id" : "count", "delayTime" : "count", "target" : "sum"})

In [ ]:
train_df.groupby("planArrival").agg({"id" : "count", "delayTime" : "count"})

In [ ]:
test_df[test_df["planArrival"] == "08:01"].groupby("date").agg({"id" : "count" , "delayTime" : "count"}).query("delayTime != 0").index.to_list()
# 20200124
# 20200213
# 20200309
# 20200528
# 20200623
# 20200923
# 20201013

In [ ]:
for date in test_df[test_df["planArrival"] == "08:01"].groupby("date").agg({"id" : "count" , "delayTime" : "count"}).query("delayTime == 0").index.to_list():
    if date in train_df["date"].unique():
        print(date)

In [ ]:
test_df[test_df["planArrival"] == "08:01"].groupby("date").agg({"id" : "count" , "delayTime" : "count"}).query("delayTime = 0").index.to_list()

In [10]:
pd.concat([train_df["date"], test_df["date"]])

0          20191204
1          20191204
2          20191204
3          20191204
4          20191204
             ...   
1259224    20201021
1259225    20201021
1259226    20201021
1259227    20201021
1259228    20201021
Name: date, Length: 2748114, dtype: int64

In [11]:
train_df[["delayTime"]]

,delayTime
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
1488880,NaN
1488881,NaN
1488882,NaN
1488883,NaN


In [14]:
train_df["lineName"][train_df["date"] == 20191204] = "F"
train_df

/opt/anaconda3/envs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,planArrivalNumeric,firstDelayTime
0,0,20191204,F,1,AFGKvJ,Rlfq,06:11,0.0,611,0.0
1,1,20191204,F,1,AFGKvJ,coZB,06:13,0.0,613,0.0
2,2,20191204,F,1,AFGKvJ,LMww,06:16,0.0,616,0.0
3,3,20191204,F,1,AFGKvJ,VNyR,06:18,0.0,618,0.0
4,4,20191204,F,1,AFGKvJ,jhlV,06:20,0.0,620,0.0
...,...,...,...,...,...,...,...,...,...,...
1488880,1488880,20201022,D,2,zbKwea,BCRD,18:47,NaN,1847,0.0
1488881,1488881,20201022,D,2,zbKwea,AVjc,18:51,NaN,1851,0.0
1488882,1488882,20201022,D,2,zbKwea,uYlv,18:53,NaN,1853,0.0
1488883,1488883,20201022,D,2,zbKwea,wwYD,18:59,NaN,1859,0.0
